<a href="https://colab.research.google.com/github/StankoDiego/SOA_EA_2/blob/main/HPC/Stanko_Diego_ejercicio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introducción

Se va a implementar un algoritmo para resolver la suma de matrices de dimension N*N. 

Se implementaran dos algoritmos, uno ejecuta y resuelve el calculo de manera secuencial sobre el mismo procesador y el segundo implementa la suma de los terminos de la matriz de forma paralela utilizando GPU. Para esto se implementan hilos en dos dimensiones, a partir de un calulo en el kernel podemos acceder a cada elemento de la matriz para poder realizar la operacion.


#1.1 Explicación del algoritmo

La suma de matrices[1] es una operación lineal que consiste en unificar los elementos de dos o más matrices que coincidan en posición dentro de sus respectivas matrices y que estas tengan el mismo orden.

En otras palabras, el sumatorio de una o más matrices es la unión de los elementos que tengan la misma posición dentro de las matrices y que estas tengan el mismo orden.

![](https://drive.google.com/uc?export=view&id=16M9RJRRsQELQqiC15qRqNC_bdQBfMc1_)

Para sumar dos matrices se debe verificar que el orden de estas sean iguales.

#2. Armado del ambiente

##2.1 Armado del ambiente en CPU

No se requiere la ejecucion previa de algun comando

##2.1 Armado del ambiente en GPU

1. Ir a Entornto de ejecucion -> Cambiar tipo de entorno de ejecucion y seleccionar la opcion GPU.
2. Se requiere la instalación del modulo de Cuda para Python.

In [1]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 8.9MB/s 
     |████████████████████████████████| 71kB 11.7MB/s 
     |████████████████████████████████| 81kB 12.1MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621146 sha256=321fc1a8c56f80983a5baae3516366e6d6af350a069312ce981e695cc244a068
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=c1a63b1052883256446a1e081180cb8df15239e6af4fd43a2468225260694dda
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


#3. Desarrollo

##3.1 Desarrollo en CPU

In [4]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }
tamMatriz =  17#@param {type: "number"}
# --------------------------------------------

import numpy
import time
import sys

tiempoTotal = time.time() * 1000
try: 
  matriz_A = numpy.random.randint(5000, size=(tamMatriz, tamMatriz))
  matriz_A = matriz_A.astype(numpy.int32())
  matriz_B = numpy.random.randint(5000, size=(tamMatriz, tamMatriz))
  matriz_B = matriz_B.astype(numpy.int32())
except Exception:
  sys.exit("El tamaño no puede ser negativo")

print("---------------------------------------------------")
print("Matriz A:\n",matriz_A)
print("---------------------------------------------------")
print("Matriz B:\n",matriz_B)
print("---------------------------------------------------")

tiempoBucle = time.time() * 1000
matriz_R = numpy.zeros((tamMatriz, tamMatriz))  
matriz_R = matriz_R.astype(numpy.int32())

for i in range (0, tamMatriz):
  for j in range(0, tamMatriz):
    matriz_R[i][j] = matriz_A[i][j] + matriz_B[i][j]

print("Matriz R:\n",matriz_R)
print("---------------------------------------------------")

tiempoTotal = (time.time() * 1000) - tiempoTotal
tiempoBucle = (time.time() * 1000) - tiempoBucle
print("--------------------------------------------------")
print("Tiempo del bucle: " , tiempoBucle , "[ms.]")
print("Tiempo del total: " , tiempoTotal , "[ms.]")

---------------------------------------------------
Matriz A:
 [[3798 2411 3083 1683 3484 2709  762 3261 1082 4599 3985 1842 3087 2896
  3666 2448 2123]
 [2641 1479 3870 2053    8 3159  609  446 1596 4852 2166 4633 4410 4614
  2513 1623  111]
 [3286 3732 2079  818 4457 1922 4692 4011  882 1963 3285 4452 3897 1870
  3709 3833 1713]
 [3074 4141 3043 1300 4277  143 3623 2324 3774 2651 3472 1567 3897 2967
  2922 3394 1122]
 [2554 4850 2757  774 4643  773 3013 3547 4928 4354   15  207 2456 2534
  2805  777 2147]
 [4855 3814 2122 2354  308 2270 3476 2204  516 4463 3206 1080  780 2825
  2517 2576 1225]
 [1324 3214 2031  502 2822 2291 1664 1276 2988 4218 4572 2345 3101 3657
   409  768 2733]
 [4767 2714 3754 4929 2684 4797 2324 4543 1071 3055 4574  952  625 2637
  4581 3150 2336]
 [2165 2203 4760 1338 1408 3190 2952   69   14 2420 1326 2220 1188  741
  3913 1029 4958]
 [4946 2619 1455 2911 4191 1904 2145 3317 3726 3596 1985 1991 4698 3540
  2593 4017 1041]
 [ 900 1003 2915  622  787 4764  346 

##3.2 Desarrollo en GPU

In [5]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }
tamMatriz =  17#@param {type: "number"}
# --------------------------------------------

import time

import numpy
import sys
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

tiempoTotal = (time.time()*1000)

try: 
  matriz_A_cpu = numpy.random.randint(5000, size=(tamMatriz, tamMatriz))
  matriz_A_cpu = matriz_A_cpu.astype(numpy.int32())
  matriz_B_cpu = numpy.random.randint(5000, size=(tamMatriz, tamMatriz))
  matriz_B_cpu = matriz_B_cpu.astype(numpy.int32())
except Exception:
  sys.exit("El tamaño de la matriz no puede ser negativo")

try:  
  matriz_R_cpu = numpy.zeros((tamMatriz, tamMatriz)) 
  matriz_R_cpu = matriz_R_cpu.astype(numpy.int32()) 
  matriz_A_gpu = cuda.mem_alloc(matriz_A_cpu.nbytes)
  matriz_B_gpu = cuda.mem_alloc(matriz_B_cpu.nbytes)
  matriz_R_gpu = cuda.mem_alloc(matriz_R_cpu.nbytes)
  cuda.memcpy_htod(matriz_A_gpu, matriz_A_cpu)
  cuda.memcpy_htod(matriz_B_gpu, matriz_B_cpu)
  cuda.memcpy_htod(matriz_R_gpu, matriz_R_cpu)
except Exception:
  sys.exit("Error de asignacion de memoria para gpu")

print("---------------------------------------------------")
print("Matriz A:\n",matriz_A_cpu)
print("---------------------------------------------------")
print("Matriz B:\n",matriz_B_cpu)
print("---------------------------------------------------")

module = SourceModule("""
__global__ void matrixMultiplicationKernel(float* A, float* B, float* R,
                                           int N
                                           ) {

    int x = blockIdx.y*blockDim.y+threadIdx.y;
    int y = blockIdx.x*blockDim.x+threadIdx.x;

    if(y < N){
      R[x+ y*N] = A[x+ y*N]  +  B[x+ y*N];
    }
}
""")
kernel = module.get_function("matrixMultiplicationKernel")

dim_hilo_x = 16
dim_bloque_x = numpy.int( (tamMatriz + dim_hilo_x-1) / dim_hilo_x )

dim_hilo_y = 16
dim_bloque_y = numpy.int( (tamMatriz + dim_hilo_y-1) / dim_hilo_y )

print( "Thread: [", dim_hilo_x, ",", dim_hilo_y, " ], Bloque : [", dim_bloque_x, ",", dim_bloque_y, "]" )
print( "Total de Thread: [", dim_hilo_x*dim_bloque_x, ",", dim_hilo_y*dim_bloque_y, " ]", " = ", dim_hilo_x*dim_bloque_x*dim_hilo_y*dim_bloque_y )

tiempo_kernel = time.time() * 1000

kernel(matriz_A_gpu, matriz_B_gpu, matriz_R_gpu,
       numpy.int32(tamMatriz),        
       block=( dim_hilo_x, dim_hilo_y, 1), 
       grid=(dim_bloque_x, dim_bloque_y,1)
       )

tiempo_kernel = (time.time() *1000) - tiempo_kernel

cuda.memcpy_dtoh(matriz_R_cpu, matriz_R_gpu)
tiempoTotal = (time.time() * 1000) - tiempoTotal
print("---------------------------------------------------")
print("Matriz resultado:\n",matriz_R_cpu)
print("---------------------------------------------------")
print("Tiempo TOTAL: ", tiempoTotal, "[ms]" )
print("Tiempo GPU  : ", tiempo_kernel, "[ms]" )

---------------------------------------------------
Matriz A:
 [[2559 4484 3477 4563 4488 2212  891 4625 3978 4475 4407 4662 4024 1394
  3138  306 3014]
 [3318 3786 3883 3757 2097 1190  791 1270 2106 4506  522 1074 4052 1675
  1551 4272 4509]
 [1163 2937  867 3913 2482 3457 4545 4467 3125 3278 1188 1762  912  120
  3411 4678 1340]
 [4729 2398 3860 4828 3657  287 2098 2911  194 1090  790 4539 3395 2737
  4060 1412 4465]
 [4730 3557 2498   86 4785 3570  410 2223 2627 1329 1697 3096  177  374
  3069 1958 2710]
 [2306 2302 4342 4113  393 2964 4116 1272 2056 3713 1529  660 2360  843
   408  282 1934]
 [ 783 4283 3637 4071 4622 4285 3169 2401  895 3857 3844  174 2630 1373
  4225  699 1222]
 [ 890  169 1998 2775 1645 4777 1675 1837 1852 3410 2293 2103 1984 3998
  3017  830 2748]
 [3049  873 3657 3786 2154 1471 4201 2825 3431   96  582 3480  846 2191
  3218 3120 4660]
 [ 837 1078 3529 2179 1810 2449 1747 2546 3092 1618 4948 2109  197 3996
  4915  119  755]
 [ 726 1219 3932 1164 2737 2269 4339 

#4. Tabla de pasos

##4.1 Tabla de pasos CPU

Tabla de ejecucion de forma secuencial
>Procesador | Funcion | Detalle
>--- | --- | ----
>CPU	|@param              |Lectura del tamaño de matriz.
>CPU	|import              |Se importa los diferentes modulos a utilizar.
>CPU	|tiempoTotal         |Se obtiene el tiempo actual para futuras mediciones.
>CPU	|numpy.random.randint|Inicializacion de matrices.
>CPU	|tiempoBucle	       |Se toma el tiempo inicial del algoritmo presentado.
>CPU	|for…	for            |Se realiza el algoritmo.
>CPU	|tiempoBucle	       |Toma el tiempo final del algoritmo presentado.
>CPU	|tiempoTotal	       |Toma el tiempo final de la ejecucion.
>CPU	|print()	           |Se informan los datos.

##4.1 Tabla de pasos GPU

Tabla de ejecucion en multiples hilos de ejecucion
>Procesador | Funcion | Detalle
>--- | --- | ----
>CPU	|@param              |Lectura del tamaño de matriz.
>CPU	|import              |Se importa los diferentes modulos a utilizar.
>CPU	|tiempoTotal         |Se obtiene el tiempo actual para futuras mediciones.
>CPU	|numpy.random.randint|Inicializacion de matrices.
>CPU	|numpy.zeros         |Inicializacion de matriz de resultado.
>**GPU**  |cuda.mem_alloc        |Reserva de memoria de GPU.
>**GPU**  |cuda.memcpy_htod      |Copia de memoria de CPU a GPU.
>CPU	|SourceModule        |Definición de código del kernel.
>CPU	|module.get_function        |Generación de función del Kernel de GPU.
>CPU	|dim_hilo_x, dim_hilo_y	    | Calcula las dimensiones para la ejecuciòn de 2D.
>CPU	|tiempo_kernel	       |Toma el tiempo inicial del algoritmo presentado.
>**GPU**	|kernel()            |Ejecución del kernel
>CPU	|tiempo_kernel	       |Toma el tiempo final del algoritmo presentado.
>CPU	|tiempoTotal	       |Toma el tiempo final de la ejecucion.
>CPU	|print()	           |Se informan los datos.

#5. Conclusiones

El algoritmo no presenta mucha dificultad, lo interesante es que el primer algoritmo posee una complejidad O(N^2) y su rendimiento empeorara a medida que las dimensiones de la matriz sean mayores. Con la implementación paralela aumenta la eficiencia frente a la ejecución secuencial.

#6. Bibliografía

[1].Suma de matrices [matrices](https://economipedia.com/definiciones/suma-de-matrices.html)